# Traffic Light Training
---

#### Resources: 
1. [Training on Google Cloud](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/running_pets.md)
2. [Training the model (medium-WuStangDan)](https://medium.com/@WuStangDan/step-by-step-tensorflow-object-detection-api-tutorial-part-4-training-the-model-68a9e5d5a333)
3. [Sample .config file](https://github.com/tensorflow/models/blob/master/research/object_detection/faster_rcnn_inception_resnet_v2_atrous_oid.config)
4. [Reading TFRecord file](http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/)
5. [COCO Trained Models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)


## Environment Preparation

In [49]:
# imports
import numpy as np
import os
import sys
from PIL import Image
import tensorflow as tf
import skimage.io as io
from matplotlib import pyplot as plt

# This is needed to display the images.
%matplotlib inline

# Dataset paths
PATH_TO_GREEN_IMGS = "../../dataset/images_for_training/green/"
PATH_TO_RED_IMGS = "../../dataset/images_for_training/red/"
PATH_TO_YELLOW_IMGS = "../../dataset/images_for_training/yellow/"
PATH_TO_XML = "../../dataset/images_for_training/"
OUTPUT_PATH = os.path.dirname("../../output_data/")

# Place TF Models repository on the same directory (https://github.com/tensorflow/models)
sys.path.append("../../models/research/")
sys.path.append("../../models/research/slim/") # deployment
sys.path.append("../../models/research/object_detection/")

# TFrecords files
xml_train = "xml_train.tfrecords"
yaml_train = "yaml_train.tfrecords"

## Helper Functions

In [38]:
IMAGE_HEIGHT = 384
IMAGE_WIDTH = 384

def read_and_decode(filename_queue):
    
    reader = tf.TFRecordReader()

    _, serialized_example = reader.read(filename_queue)

    features = tf.parse_single_example(
      serialized_example,
      # Defaults are not specified since both keys are required.
      features={
        'height': tf.FixedLenFeature([], tf.int64),
        'width': tf.FixedLenFeature([], tf.int64),
        'image_raw': tf.FixedLenFeature([], tf.string),
        'mask_raw': tf.FixedLenFeature([], tf.string)
        })

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    annotation = tf.decode_raw(features['mask_raw'], tf.uint8)
    
    height = tf.cast(features['height'], tf.int32)
    width = tf.cast(features['width'], tf.int32)
    
    image_shape = tf.stack([height, width, 3])
    annotation_shape = tf.stack([height, width, 1])
    
    image = tf.reshape(image, image_shape)
    annotation = tf.reshape(annotation, annotation_shape)
    
    image_size_const = tf.constant((IMAGE_HEIGHT, IMAGE_WIDTH, 3), dtype=tf.int32)
    annotation_size_const = tf.constant((IMAGE_HEIGHT, IMAGE_WIDTH, 1), dtype=tf.int32)
    
    # Random transformations can be put here: right before you crop images
    # to predefined size. To get more information look at the stackoverflow
    # question linked above.
    
    resized_image = tf.image.resize_image_with_crop_or_pad(image=image,
                                           target_height=IMAGE_HEIGHT,
                                           target_width=IMAGE_WIDTH)
    
    resized_annotation = tf.image.resize_image_with_crop_or_pad(image=annotation,
                                           target_height=IMAGE_HEIGHT,
                                           target_width=IMAGE_WIDTH)
    
    
    images, annotations = tf.train.shuffle_batch( [resized_image, resized_annotation],
                                                 batch_size=2,
                                                 capacity=30,
                                                 num_threads=2,
                                                 min_after_dequeue=10)
    
    return images, annotations

# Check TF File

In [35]:
### Retrieve and Check TF records file ###
tfrecords_filename = 'xml_train.tfrecords'

filename_queue = tf.train.string_input_producer(
    [tfrecords_filename], num_epochs=10)

# Even when reading in multiple threads, share the filename
# queue.
image, annotation = read_and_decode(filename_queue)

# The op for initializing the variables.
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

with tf.Session()  as sess:
    
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # Let's read off 3 batches just for example
    for i in range(1):
    
        img, anno = sess.run([image, annotation])
        print(img[0, :, :, :].shape)
        
        print('current batch')
        
        # We selected the batch size of two
        # So we should get two image pairs in each batch
        # Let's make sure it is random

        io.imshow(img[0, :, :, :])
        io.show()

        io.imshow(anno[0, :, :, 0])
        io.show()
        
        io.imshow(img[1, :, :, :])
        io.show()

        io.imshow(anno[1, :, :, 0])
        io.show()
        
    
    coord.request_stop()
    coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.NotFoundError'>, xml_train.tfrecords
	 [[Node: ReaderReadV2 = ReaderReadV2[_device="/job:localhost/replica:0/task:0/cpu:0"](TFRecordReaderV2, input_producer)]]


INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.NotFoundError'>, xml_train.tfrecords
	 [[Node: ReaderReadV2 = ReaderReadV2[_device="/job:localhost/replica:0/task:0/cpu:0"](TFRecordReaderV2, input_producer)]]


OutOfRangeError: RandomShuffleQueue '_130_shuffle_batch_20/random_shuffle_queue' is closed and has insufficient elements (requested 2, current size 0)
	 [[Node: shuffle_batch_20 = QueueDequeueManyV2[component_types=[DT_UINT8, DT_UINT8], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch_20/random_shuffle_queue, shuffle_batch_20/n)]]

Caused by op 'shuffle_batch_20', defined at:
  File "/Users/eric/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/eric/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/eric/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/eric/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/eric/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
 

## Import COCO dataset

In [ ]:
# pick and download a model

model_name = 'faster_rcnn_resnet101_coco_2017_11_08'


# Assume fc_2nd_last is the 2nd_last fully connected layer in your network and nb_classes is the number of classes in your new dataset.
#shape = (fc_2nd_last.get_shape().as_list()[-1], nb_classes)
#fc_last_W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
#fc_last_b = tf.Variable(tf.zeros(nb_classes))
#logits = tf.nn.xw_plus_b(fc_2nd_last, fc_last_W, fc_last_b)


In [ ]:
# Configure labels

In [48]:
# Train
#train_file = '../../models/research/object_detection/train.py'
#%run ../../models/research/object_detection/train.py
